## Does PUBG contain "bots"?

This notebook is mainly for learning purpose, just tried to apply some knowledge to a problem. I dont know how accurate my analysis is, but you learn from your mistakes and experiences. 
The goal is to check or confirm a hypothesis or an assumption that "bots" exists in PUBG which every player knows.
"Bots" are those free frags which come out of no where and stare at you only to be killed in seconds.

The conditions where we find bots are:
1. Lonely places where there's no enemy.
2. Crowded areas with lots of enemy.

Credits to https://www.kaggle.com/skihikingkevin/pubg-match-deaths#deaths.zip for the dataset.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
kills = pd.read_csv("../input/deaths/kill_match_stats_final_0.csv")
matches = pd.read_csv("../input/aggregate/agg_match_stats_0.csv")

Let's have a look at the matches dataframe. 
The columns are as follows:

match_id: The unique match id as generated by pubg.op.gg. You can do a join of this with the kills data to view all information
date: The date and time the match took place
game_size: The total number of teams that were in the game
match_mode: whether the game was played in first-person (fpp) or third-person (tpp)
party_size: The maximum number of players per team. e.g 2 implies it was a duo queue system
player_name: Name of the player
team_id: The team id that the player belonged to
team_placement: The final rank of the team within the match
player_kills: Number of kills the player has scored
player_dbno: Number of knockdowns the player has scored
player_assists: Number of assists the player has scored
player_dmg: Total Hitpoint that the player has dealt
player_dist_ride: Total distance (in metres?) that the player has travelled in a vehicle
player_dist_walk: Total distance (in metres?) that the player has travelled on foot

In [ ]:
matches.shape
matches.head()

Columns: killed_by, killer_name, killer_placement, killer_position_x, killer_position_y, map, match_id, time, victim_name, victim_placement, victim_position_x, victim_position_y

In [ ]:
kills.head()


In [ ]:
kills.shape

In [ ]:
#Lets just start with solo games. So we remove other games.
match_stats = matches[matches["party_size"]==1].drop_duplicates()

In [ ]:
#Making a list of all the games we are interested in.
match_list = match_stats["match_id"].values


Now our aim is to find or try to differentiate "bots" from the actual players. One big assumption and mostly true in practical sense is that "bots" dont kill much rather "zero". 
So to find such "bots", we only look at players who have zero kills. 

In [ ]:
match_zero_kills = match_stats[match_stats["player_kills"]==0]
match_zero_kills.shape

In [ ]:
#Heres how our new dataframe with players only with zero kills looks like
match_zero_kills.head()

In [ ]:
#Now since the data is too huge, lets just select a match randomly and analyze it further
#It contains all playrs from the game.
selected_match = matches[matches["match_id"]==match_list[4]]
selected_match.shape

In [ ]:
#Now since the data is too huge, lets just select a match randomly and analyze it further. 
#This however contains only players with zero kills
selected_game_zero = match_zero_kills[match_zero_kills["match_id"]== match_list[4]]
selected_game_zero.shape

In [ ]:
selected_game_zero.head()

In [ ]:
#Lets take a look at the kill stats from the same selected game.
kills_in_selected_game = kills[kills["match_id"]==match_list[4]]
kills_in_selected_game.shape

In [ ]:
#Lets merge the kill stats and the match stats for the select game.
select_game_dataset = selected_game_zero.merge(kills_in_selected_game, left_on="player_name", right_on="victim_name", how="inner")
select_game_dataset.shape

In [ ]:
#This contains all the players, not just the ones with zero kills.

selected_game_dataset_all = selected_match.merge(kills_in_selected_game, left_on="player_name", right_on="victim_name", how="inner")
selected_game_dataset_all.shape

In [ ]:
#This is quite intuitive space. A "bot" would never drive a car or survive for a long time, cause they usually get killed as soon as they spawn.
#Dropping players who have driven a vehicle
select_game_dataset = select_game_dataset[select_game_dataset["player_dist_ride"] == 0]
select_game_dataset = select_game_dataset[select_game_dataset["player_survive_time"] < 300]
select_game_dataset.shape

In [ ]:
select_game_dataset.head()

In [ ]:
features = ["match_id_x", "player_dist_ride", "player_dist_walk", "player_dmg","player_survive_time","team_id", "killed_by","killer_name", "killer_placement", 
            "killer_position_x", "killer_position_y", "time", "victim_name", "victim_placement", "victim_position_x",
            "victim_position_y"]
len(features)

In [ ]:
selected_game_bots = select_game_dataset[features]
selected_game_bots.shape

one last thingg, again intuitive and needs to have some domain knowledge. "killed_by" contains some ways by which the "bots" wont die.
They are - "Down and Out" , 'Bluezone' , 'death.RedZoneBomb_C' , 'Falling' , 'Drown' , 'RedZone'


In [ ]:
#removed_features = ["Down and Out" , 'Bluezone' , 'death.RedZoneBomb_C' , 'Falling' , 'Drown' , 'RedZone']
selected_game_bots = selected_game_bots[~selected_game_bots.killed_by.str.contains("Down and Out" , 'Bluezone')]
selected_game_bots = selected_game_bots[~selected_game_bots.killed_by.str.contains('death.RedZoneBomb_C' , 'Falling')]
selected_game_bots = selected_game_bots[~selected_game_bots.killed_by.str.contains('Drown' , 'RedZone')]

selected_game_bots.shape

In [ ]:
#This is how the "bots" were killed.
selected_game_bots.killed_by.value_counts()

### Let's plot all the victims that have zero kills

In [ ]:
import matplotlib.pyplot as plt
from scipy.misc import imread
img = imread("../input/erangel.jpg")
implot = plt.imshow(img,aspect='auto', extent=[0, 600000, 0, 700000])

# put a blue dot at (10, 20)
plt.plot(selected_game_bots["victim_position_x"], selected_game_bots["victim_position_y"], "o")
#plt.plot(all_dataset["victim_position_x"], all_dataset["victim_position_y"],  "ro")


plt.show()

In [ ]:
selected_game_dataset_all = selected_game_dataset_all[features]
selected_game_dataset_all.shape

### Plotting all the victims.

In [ ]:
import matplotlib.pyplot as plt
img = imread("../input/erangel.jpg")
implot = plt.imshow(img,aspect='auto', extent=[0, 600000, 0, 700000])

# put a blue dot at (10, 20)
plt.plot(selected_game_dataset_all["victim_position_x"], selected_game_dataset_all["victim_position_y"], "o")
#plt.plot(all_dataset["killer_position_x"], all_dataset["killer_position_y"], "ro")


plt.show()

### Plotting all the killers

In [ ]:

img = imread("../input/erangel.jpg")
implot = plt.imshow(img,aspect='auto', extent=[0, 600000, 0, 700000])

# put a blue dot at (10, 20)
#plt.plot(all_dataset["victim_position_x"], all_dataset["victim_position_y"], "o")
plt.plot(selected_game_dataset_all["killer_position_x"], selected_game_dataset_all["killer_position_y"], "ro")


plt.show()

## Conclusion

As seen from the plots and some PUBG knowledge, "bots" spawn on two occassions:
1. When there are no enemies around for a while. 
2. If a place is crowded with enemies, some bots can spawn.

Analysis:
Looking at the plots, the top right corner might satisify condition #1. However, there's no way to confirm that was a bot.
But as mentioned earlier, the goal isnt to prove bots exists but to put up a theory that they do.

PS: This was my first ever attempt at a blog post, comment and let me know where I went wrong or how can I do better. 
Thank You!